In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import re
import os,sys

import kbij_helper as kbijh
kbij = kbijh.KBIJ()

import warnings
warnings.filterwarnings('ignore')
autostats_lib = "../../../autostats"
if autostats_lib not in sys.path:
    sys.path.append(autostats_lib)

from utilities.data_storage import DataStorage
import time

In [2]:
%load_ext google.cloud.bigquery
from google.cloud import bigquery
DataStorage(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

from IPython.display import clear_output

In [3]:
def get_key(my_dict, val):
    keys_list = []
    for key, value in my_dict.items(): 
         if val == value: 
             keys_list.append(key)
  
    return keys_list

In [4]:
date = "2020-02-01"
prev_date = "2020-01-01"

In [5]:
datem = pd.to_datetime(date)
datem = datem.strftime("%Y%m")
print(datem)

202002


In [6]:
pathname = 'data'+datem
dirname = './'+pathname

In [7]:
raw_filename_F01 = dirname+'/F01'+datem+'.csv'
raw_filename_D01 = dirname+'/D01'+datem+'.csv'

In [8]:
filtered_filename_F01 = dirname+'/F01'+datem+'_new_filtered.csv'
filtered_filename_D01 = dirname+'/D01'+datem+'_new_filtered.csv'

In [9]:
prev_datem = pd.to_datetime(prev_date)
prev_datem = prev_datem.strftime("%Y%m")
print(prev_datem)

202001


In [10]:
prev_pathname = 'data'+prev_datem
prev_dirname = './'+prev_pathname

In [11]:
prev_filtered_filename_F01 = prev_dirname+'/F01'+prev_datem+'_new_filtered.csv'
prev_filtered_filename_D01 = prev_dirname+'/D01'+prev_datem+'_new_filtered.csv'

In [12]:
F01_dtype = {'Kode Jenis Pelapor': 'object',
 'Kode Pelapor': 'object',
 'Tahun Bulan Data': 'object',
 'Kode Jenis Fasilitas': 'object',
 'Nomor Rekening Fasilitas': 'object',
 'Nomor Rekening Lama Fasilitas': 'object',
 'Nomor CIF Debitur': 'object',
 'Kode Sifat Kredit atau Pembiayaan': 'str',
 'Kode Jenis Kredit atau Pembiayaan': 'str',
 'Kode Akad Kredit atau Akad Pembiayaan': 'object',
 'Nomor Akad Awal': 'object',
 'Tanggal Akad Awal': 'object',
 'Nomor Akad Akhir': 'object',
 'Tanggal Akad Akhir': 'object',
 'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan': 'int64',
 'Tanggal Awal Kredit atau Pembiayaan': 'object',
 'Tanggal Mulai': 'object',
 'Tanggal Jatuh Tempo': 'object',
 'Kode Kategori Debitur': 'object',
 'Kode Jenis Penggunaan': 'str',
 'Kode Orientasi Penggunaan': 'str',
 'Kode Sektor Ekonomi': 'object',
 'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan': 'object',
 'Nilai Proyek': 'object',
 'Kode Valuta': 'object',
 'Suku Bunga atau Imbalan': 'float64',
 'Jenis Suku Bunga atau Imbalan': 'int64',
 'Kredit atau Pembiayaan Program Pemerintah': 'object',
 'Plafon Awal': 'float64',
 'Plafon': 'float64',
 'Realisasi atau Pencairan Bulan Berjalan': 'float64',
 'Denda': 'float64',
 'Baki Debet': 'float64',
 'Nilai dalam Mata Uang Asal': 'object',
 'Kode Kualitas Kredit atau Pembiayaan': 'str',
 'Tanggal Macet': 'object',
 'Kode Sebab Macet': 'str',
 'Tunggakan Pokok': 'int64',
 'Tunggakan Bunga atau Imbalan': 'int64',
 'Jumlah Hari Tunggakan': 'int64',
 'Frekuensi Tunggakan': 'int64',
 'Frekuensi Restrukturisasi': 'int64',
 'Tanggal Restrukturisasi Awal': 'object',
 'Tanggal Restrukturisasi Akhir': 'object',
 'Kode Cara Restrukturisasi': 'object',
 'Kode Kondisi': 'object',
 'Tanggal Kondisi': 'object',
 'Keterangan': 'object',
 'Kode Kantor Cabang': 'object',
 'Operasi Data': 'object',
 'Status delete': 'object',
 'Create Date': 'str',
 'Update Date': 'str',
 'Approved Date': 'str',
 'userId': 'object',
 'orderId': 'object',
 'status': 'object'}

D01_dtype = {'Kode Jenis Pelapor': 'object',
 'Kode Pelapor': 'object',
 'Tahun Bulan Data': 'object',
 'Nomor CIF Lama Debitur': 'object',
 'Nomor CIF Debitur': 'object',
 'Jenis Identitas': 'int64',
 'Nomor Identitas': 'object',
 'Nama Sesuai Identitas': 'object',
 'Nama Lengkap': 'object',
 'Kode Status Pendidikan atau Gelar Debitur': 'object',
 'Jenis Kelamin': 'object',
 'Tempat Lahir': 'object',
 'Tanggal Lahir': 'object',
 'Nama Gadis Ibu Kandung': 'object',
 'NPWP': 'object',
 'Alamat': 'object',
 'Kelurahan': 'object',
 'Kecamatan': 'object',
 'Kode Kabupaten atau Kota': 'object',
 'Kode Pos': 'object',
 'Nomor Telepon': 'object',
 'Nomor Telepon Seluler': 'object',
 'Alamat email': 'object',
 'Kode Negara Domisili': 'object',
 'Kode Pekerjaan': 'object',
 'Tempat Bekerja': 'object',
 'Kode Bidang Usaha Tempat Bekerja': 'object',
 'Alamat Tempat Bekerja': 'object',
 'Kode Golongan Debitur': 'object',
 'Status Perkawinan Debitur': 'int64',
 'Kode Kantor Cabang': 'object',
 'Operasi Data': 'object',
 'Status delete': 'object',
 'Create Date': 'str',
 'Update Date': 'str',
 'Approved Date': 'str',
 'userId': 'object',
 'orderId': 'object',
 'status': 'object'}

# Data Preparation

In [13]:
DataStorage.download_notebooks_data(raw_filename_F01, 'kbij/'+pathname, 'f01'+datem+'_new_filtered.csv')

Blob notebooks/kbij/data202002/f01202002_new_filtered.csv version LATEST downloaded to ./data202002/F01202002.csv.


In [14]:
DataStorage.download_notebooks_data(raw_filename_D01, 'kbij/'+pathname, 'd01'+datem+'_new_filtered.csv')

Blob notebooks/kbij/data202002/d01202002_new_filtered.csv version LATEST downloaded to ./data202002/D01202002.csv.


In [15]:
F01new = pd.read_csv(raw_filename_F01, dtype=F01_dtype)
D01new = pd.read_csv(raw_filename_D01, dtype=D01_dtype)

Change Kode Pelapor to 810038

In [16]:
F01new['Kode Pelapor'] = F01new['Kode Pelapor'].replace("900013","810038")
D01new['Kode Pelapor'] = D01new['Kode Pelapor'].replace("900013","810038")

In [17]:
D01new['Kode Pelapor']

0         810038
1         810038
2         810038
3         810038
4         810038
5         810038
6         810038
7         810038
8         810038
9         810038
10        810038
11        810038
12        810038
13        810038
14        810038
15        810038
16        810038
17        810038
18        810038
19        810038
20        810038
21        810038
22        810038
23        810038
24        810038
25        810038
26        810038
27        810038
28        810038
29        810038
           ...  
119306    810038
119307    810038
119308    810038
119309    810038
119310    810038
119311    810038
119312    810038
119313    810038
119314    810038
119315    810038
119316    810038
119317    810038
119318    810038
119319    810038
119320    810038
119321    810038
119322    810038
119323    810038
119324    810038
119325    810038
119326    810038
119327    810038
119328    810038
119329    810038
119330    810038
119331    810038
119332    810038
119333    8100

In [18]:
F01new['Kode Pelapor']

0         810038
1         810038
2         810038
3         810038
4         810038
5         810038
6         810038
7         810038
8         810038
9         810038
10        810038
11        810038
12        810038
13        810038
14        810038
15        810038
16        810038
17        810038
18        810038
19        810038
20        810038
21        810038
22        810038
23        810038
24        810038
25        810038
26        810038
27        810038
28        810038
29        810038
           ...  
144553    810038
144554    810038
144555    810038
144556    810038
144557    810038
144558    810038
144559    810038
144560    810038
144561    810038
144562    810038
144563    810038
144564    810038
144565    810038
144566    810038
144567    810038
144568    810038
144569    810038
144570    810038
144571    810038
144572    810038
144573    810038
144574    810038
144575    810038
144576    810038
144577    810038
144578    810038
144579    810038
144580    8100

Check for Data With Denda>0 AND Kode Kondisi='01'

In [19]:
F01new[(F01new['Denda']>0) & (F01new['Kode Kondisi']=='01')][['orderId','Denda','Kode Kondisi']]

,orderId,Denda,Kode Kondisi
36178,ATH-9ANCE5H8,50000.0,01


In [20]:
F01new["Denda"][(F01new['Denda']>0) & (F01new['Kode Kondisi']=='01')]=0

In [21]:
F01new[(F01new['Denda']>0) & (F01new['Kode Kondisi']=='01')][['orderId','Denda','Kode Kondisi']]

,orderId,Denda,Kode Kondisi


Test

In [30]:
F01new[['orderId','Tanggal Jatuh Tempo','Tanggal Mulai','Kode Kondisi']][F01new['Kode Kondisi']=='01']

,orderId,Tanggal Jatuh Tempo,Tanggal Mulai,Kode Kondisi
172,ATH-12GPBT24,20200221,20200121,01
422,ATH-14K125HK,20200204,20200104,01
591,ATH-15R3JTVJ,20200218,20200118,01
600,ATH-15TDUNA7,20190307,20190207,01
870,ATH-17XXYM99,20200216,20200116,01
929,ATH-18CSGWKH,20200228,20200128,01
1046,ATH-19AKDDAD,20200328,20200128,01
1521,ATH-1CZA3KU5,20180816,20180716,01
1740,ATH-1EMZ52RJ,20200228,20200128,01
1974,ATH-1GGRGRM6,20200428,20200128,01


F01 - Tanggal Jatuh Tempo tidak terisi

In [37]:
F01new[F01new['Tanggal Jatuh Tempo'].isna()][['orderId','Tanggal Jatuh Tempo','Tanggal Mulai','Kode Kondisi']]

,orderId,Tanggal Jatuh Tempo,Tanggal Mulai,Kode Kondisi


In [34]:
# 20180208
F01new['Tanggal Mulai'][F01new['orderId']=='ATH-EG3N2E52'] = '20180208'
F01new['Tanggal Jatuh Tempo'][F01new['orderId']=='ATH-EG3N2E52'] = '20180308'

In [35]:
F01new[F01new['orderId']=='ATH-EG3N2E52'][['orderId','Tanggal Jatuh Tempo','Tanggal Mulai','Kode Kondisi']]

,orderId,Tanggal Jatuh Tempo,Tanggal Mulai,Kode Kondisi
58522,ATH-EG3N2E52,20180308,20180208,01


F01 - Tanggal Mulai tidak terisi

In [36]:
F01new[F01new['Tanggal Mulai'].isna()][['orderId','Tanggal Jatuh Tempo','Tanggal Mulai']]

,orderId,Tanggal Jatuh Tempo,Tanggal Mulai


Check If KTP ID has char O/o

In [38]:
D01new['Nomor Identitas'][D01new['Nomor Identitas'].str.contains('O',case=False, regex=False)==True]

Series([], Name: Nomor Identitas, dtype: object)

Trim "Alamat e-mail" to 150 char, "Alamat Tempat Bekerja" to 300 char

In [40]:
D01new['Alamat e-mail']=D01new['Alamat e-mail'].str[-149:]
D01new['Alamat Tempat Bekerja']=D01new['Alamat Tempat Bekerja'].str[:299]

Check if there is still "Alamat e-mail" and "Alamat Tempat Bekerja" with more than 150 and 300 char each

In [41]:
D01new['Alamat e-mail'][D01new['Alamat e-mail'].str.len()>=150]

Series([], Name: Alamat e-mail, dtype: object)

In [42]:
D01new['Alamat Tempat Bekerja'][D01new['Alamat Tempat Bekerja'].str.len()>=300]

Series([], Name: Alamat Tempat Bekerja, dtype: object)

In [43]:
D01new[D01new['Nama Sesuai Identitas'].isna()]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Nomor CIF Lama Debitur,Nomor CIF Debitur,Jenis Identitas,Nomor Identitas,Nama Sesuai Identitas,Nama Lengkap,Kode Status Pendidikan atau Gelar Debitur,...,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date,Approved Date,userId,orderId,paidoffDate,status


In [44]:
F01new.to_csv(path_or_buf=raw_filename_F01, index=False)
D01new.to_csv(path_or_buf=raw_filename_D01, index=False)

In [45]:
DataStorage.upload_notebooks_data(raw_filename_F01, 'kbij/'+pathname, 'f01'+datem+'.csv')

File ./data202002/F01202002.csv version 2 uploaded to notebooks/kbij/data202002/f01202002.csv.


In [46]:
DataStorage.upload_notebooks_data(raw_filename_D01, 'kbij/'+pathname, 'd01'+datem+'.csv')

File ./data202002/D01202002.csv version 2 uploaded to notebooks/kbij/data202002/d01202002.csv.


# Load Data

In [47]:
DataStorage.download_notebooks_data(raw_filename_F01, 'kbij/'+pathname, 'f01'+datem+'.csv')

Blob notebooks/kbij/data202002/f01202002.csv version LATEST downloaded to ./data202002/F01202002.csv.


In [48]:
DataStorage.download_notebooks_data(raw_filename_D01, 'kbij/'+pathname, 'd01'+datem+'.csv')

Blob notebooks/kbij/data202002/d01202002.csv version LATEST downloaded to ./data202002/D01202002.csv.


In [49]:
F01new = pd.read_csv(raw_filename_F01, dtype=F01_dtype)
D01new = pd.read_csv(raw_filename_D01, dtype=D01_dtype)

In [50]:
print(prev_filtered_filename_F01)
print('kbij/'+prev_pathname)
print('f01'+prev_datem+'_new_filtered.csv')

./data202001/F01202001_new_filtered.csv
kbij/data202001
f01202001_new_filtered.csv


In [51]:
DataStorage.download_notebooks_data(prev_filtered_filename_F01, 'kbij/'+prev_pathname, 'f01'+prev_datem+'_new_filtered.csv')

Blob notebooks/kbij/data202001/f01202001_new_filtered.csv version LATEST downloaded to ./data202001/F01202001_new_filtered.csv.


In [52]:
DataStorage.download_notebooks_data(prev_filtered_filename_D01, 'kbij/'+prev_pathname, 'd01'+prev_datem+'_new_filtered.csv')

Blob notebooks/kbij/data202001/d01202001_new_filtered.csv version LATEST downloaded to ./data202001/D01202001_new_filtered.csv.


In [53]:
F01_prev = pd.read_csv(prev_filtered_filename_F01, dtype=F01_dtype)
D01_prev = pd.read_csv(prev_filtered_filename_D01, dtype=D01_dtype)

In [54]:
print('D01new:',D01new.shape, 'F01new:',F01new.shape)
print('D01_prev:',D01_prev.shape, 'F01_prev:',F01_prev.shape)

D01new: (119336, 40) F01new: (144583, 69)
D01_prev: (112372, 40) F01_prev: (135874, 69)


# Checking

In [55]:
F01new, D01new = kbij.checking_data(F01new, D01new, date, F01_prev=F01_prev, D01_prev=D01_prev)

Check Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data 

[CLEAR]No Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data

[DONE]
Check Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data 

[CLEAR]No Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data

[DONE]
Check Nomor CIF Debitur in D01 and F01 

[CLEAR]All Nomor CIF Debitur in D01 are exist in F01

[DONE]
Check Active Kode Kondisi but Baki Debet = 0 

[CLEAR]No Active Kode Kondisi but Baki Debet = 0

[DONE]
Check Not Active Kode Kondisi but Baki Debet > 0 

[CLEAR]No Not Active Kode Kondisi but Baki Debet > 0

[DONE]
Check Baki Debet > Plafon Awal 

[CLEAR]No Baki Debet greater than Plafon Awal

[DONE]
Check Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Macet 

[CLEAR]No Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Macet

[DONE]
Check Tahun Bulan Data 

F01: ['202002']
D01: ['202002']

[DONE]


## Check column

In [56]:
F01new[(F01new['Jumlah Hari Tunggakan']==0) & (F01new['Frekuensi Tunggakan']>0)][['orderId','Jumlah Hari Tunggakan','Frekuensi Tunggakan','Approved Date','status']]

,orderId,Jumlah Hari Tunggakan,Frekuensi Tunggakan,Approved Date,status


In [57]:
F01new[(F01new['Tunggakan Pokok']>0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='1') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi','dayPastDue']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi,dayPastDue


In [58]:
F01new[(F01new['Denda']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi','dayPastDue']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi,dayPastDue
332,ATH-13SVCYFQ,0.0,5,00,331
449,ATH-14T7HLXJ,0.0,5,00,230
567,ATH-15KDJSLQ,0.0,2,00,83
572,ATH-15LJUAP8,0.0,3,00,96
584,ATH-15PRH8ZH,0.0,5,00,425
662,ATH-168CD7PA,0.0,3,00,92
945,ATH-18J1TZ8J,0.0,5,00,223
958,ATH-18N1YSZX,0.0,5,00,486
995,ATH-18WV4LT1,0.0,5,00,268
1011,ATH-192YG3V5,0.0,5,00,188


In [59]:
F01new[(F01new['Tunggakan Bunga atau Imbalan']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='00')][['orderId','Tunggakan Bunga atau Imbalan','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi','dayPastDue']]

,orderId,Tunggakan Bunga atau Imbalan,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi,dayPastDue
20018,ATH-5LEAQ3CS,0,661000.0,5,00,442
25705,ATH-6XQVT373,0,950000.0,3,00,119
44859,ATH-B9PXR6QK,0,475000.0,3,00,115
68741,ATH-GTBSMBMB,0,458000.0,5,00,252
91142,ATH-MW7D4MYP,0,950000.0,5,00,620
119471,ATH-UCQBBQMV,0,661000.0,5,00,350
129508,ATH-WMZPNWPX,0,661000.0,5,00,354
144324,EXT-1CPE886S,0,950000.0,2,00,84
144325,EXT-1KE2RGRF,0,475000.0,2,00,68
144328,EXT-2GYRELC2,0,475000.0,5,00,192


In [60]:
F01new[(F01new['Jumlah Hari Tunggakan']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='2') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [61]:
F01new[(F01new['Frekuensi Tunggakan']==0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='2') & (F01new['Kode Kondisi']=='00')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [62]:
F01new[(F01new['Denda']>0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='02')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [63]:
F01new[(F01new['Tunggakan Bunga atau Imbalan']>0) & (F01new['Kode Kualitas Kredit atau Pembiayaan']!='1') & (F01new['Kode Kondisi']=='02')][['orderId','Denda','Kode Kualitas Kredit atau Pembiayaan','Kode Kondisi']]

,orderId,Denda,Kode Kualitas Kredit atau Pembiayaan,Kode Kondisi


In [64]:
F01new[(F01new['Kode Sebab Macet']=='99') & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='1')][['orderId','Kode Sebab Macet','Kode Kualitas Kredit atau Pembiayaan']]

,orderId,Kode Sebab Macet,Kode Kualitas Kredit atau Pembiayaan


In [65]:
F01new[(~F01new['Tanggal Macet'].isnull()) & (F01new['Kode Kualitas Kredit atau Pembiayaan']=='1')][['orderId','Tanggal Macet','Kode Kualitas Kredit atau Pembiayaan']]

,orderId,Tanggal Macet,Kode Kualitas Kredit atau Pembiayaan


# Final File to Send to KBIJ D01 and F01

Null check for CIF and NIK

In [66]:
D01new.loc[(D01new['Nomor CIF Debitur'].isna())|(D01new['Nama Lengkap'].isna())|(D01new['Tanggal Lahir'].isna())|(D01new['Nomor Identitas'].isna())]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Nomor CIF Lama Debitur,Nomor CIF Debitur,Jenis Identitas,Nomor Identitas,Nama Sesuai Identitas,Nama Lengkap,Kode Status Pendidikan atau Gelar Debitur,...,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date,Approved Date,userId,orderId,paidoffDate,status


In [67]:
D01new.drop(D01new.loc[(D01new['Nomor CIF Debitur'].isna())|(D01new['Nama Lengkap'].isna())|(D01new['Tanggal Lahir'].isna())|(D01new['Nomor Identitas'].isna())].index, inplace=True)

In [68]:
F01new.loc[F01new['Nomor CIF Debitur'].isna()]

,orderId,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,...,lateFee,balanceWithInterest,balance,dataDate,jumlah cicilan,current payment periode month,count payment,paidoffDate,status,Tunggakan Pokok


In [69]:
F01new.drop(F01new.loc[F01new['Nomor CIF Debitur'].isna()].index, inplace=True)

In [70]:
F01new[F01new['Denda'].isna()]

,orderId,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,...,lateFee,balanceWithInterest,balance,dataDate,jumlah cicilan,current payment periode month,count payment,paidoffDate,status,Tunggakan Pokok


In [71]:
F01new[F01new['Denda']==""]

,orderId,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,...,lateFee,balanceWithInterest,balance,dataDate,jumlah cicilan,current payment periode month,count payment,paidoffDate,status,Tunggakan Pokok


## Export full data dump to csv to be used in the next iteration

In [72]:
F01new.to_csv(path_or_buf=filtered_filename_F01, index=0)
D01new.to_csv(path_or_buf=filtered_filename_D01, index=0)

In [73]:
DataStorage.upload_notebooks_data(filtered_filename_F01, 'kbij/'+pathname, 'f01'+datem+'_new_filtered.csv')

File ./data202002/F01202002_new_filtered.csv version 2 uploaded to notebooks/kbij/data202002/f01202002_new_filtered.csv.


In [74]:
DataStorage.upload_notebooks_data(filtered_filename_D01, 'kbij/'+pathname, 'd01'+datem+'_new_filtered.csv')

File ./data202002/D01202002_new_filtered.csv version 2 uploaded to notebooks/kbij/data202002/d01202002_new_filtered.csv.


Export Latest Data

In [75]:
timestr = time.strftime("%Y%m%d-%H%M%S")
F01name = dirname+'/F01'+'-'+datem+'['+timestr+']'+'.txt'
D01name = dirname+'/D01'+'-'+datem+'['+timestr+']'+'.txt'
print(F01name,D01name)

./data202002/F01-202002[20201005-153524].txt ./data202002/D01-202002[20201005-153524].txt


In [76]:
F01new_sent=F01new[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
       'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
       'Kode Sifat Kredit atau Pembiayaan',
       'Kode Jenis Kredit atau Pembiayaan',
       'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
       'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
       'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
       'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
       'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
       'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
       'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
       'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
       'Jenis Suku Bunga atau Imbalan',
       'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
       'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
       'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
       'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
       'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
       'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
       'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
       'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
       'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
       'Create Date', 'Update Date']]
D01new_sent=D01new[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Nomor CIF Lama Debitur', 'Nomor CIF Debitur', 'Jenis Identitas',
       'Nomor Identitas', 'Nama Sesuai Identitas', 'Nama Lengkap',
       'Kode Status Pendidikan atau Gelar Debitur', 'Jenis Kelamin',
       'Tempat Lahir', 'Tanggal Lahir', 'Nama Gadis Ibu Kandung', 'NPWP',
       'Alamat', 'Kelurahan', 'Kecamatan', 'Kode Kabupaten atau Kota',
       'Kode Pos', 'Nomor Telepon', 'Nomor Telepon Seluler', 'Alamat e-mail',
       'Kode Negara Domisili', 'Kode Pekerjaan', 'Tempat Bekerja',
       'Kode Bidang Usaha Tempat Bekerja', 'Alamat Tempat Bekerja',
       'Kode Golongan Debitur', 'Status Perkawinan Debitur',
       'Kode Kantor Cabang', 'Operasi Data', 'Status delete', 'Create Date',
       'Update Date']]

In [77]:
D01new[D01new['Nama Sesuai Identitas']=='IBNU SIGIT SAPUTRO']

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Nomor CIF Lama Debitur,Nomor CIF Debitur,Jenis Identitas,Nomor Identitas,Nama Sesuai Identitas,Nama Lengkap,Kode Status Pendidikan atau Gelar Debitur,...,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date,Approved Date,userId,orderId,paidoffDate,status


In [78]:
print(F01new_sent.shape)
print(D01new_sent.shape)

(144583, 53)
(119336, 35)


## Export file for KBIJ Reporting

In [79]:
D01new_sent.to_csv(path_or_buf=D01name, sep='|',index =0)
F01new_sent.to_csv(path_or_buf=F01name, sep='|',index =0)

In [80]:
DataStorage.upload_notebooks_data(F01name, 'kbij/'+pathname, 'f01-'+datem+'.txt')

File ./data202002/F01-202002[20201005-153524].txt version 2 uploaded to notebooks/kbij/data202002/f01-202002.txt.


In [81]:
DataStorage.upload_notebooks_data(D01name, 'kbij/'+pathname, 'd01-'+datem+'.txt')

File ./data202002/D01-202002[20201005-153524].txt version 2 uploaded to notebooks/kbij/data202002/d01-202002.txt.


# Check Sent Data

In [82]:
F01_sent = pd.read_csv('./data202002/F01-202002[20201005-153524].txt',sep='|',dtype=F01_dtype)
D01_sent = pd.read_csv('./data202002/D01-202002[20201005-153524].txt',sep='|',dtype=D01_dtype)

In [83]:
F01_sent[F01_sent['Tanggal Mulai'].isna()]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,Kode Akad Kredit atau Akad Pembiayaan,...,Tanggal Restrukturisasi Akhir,Kode Cara Restrukturisasi,Kode Kondisi,Tanggal Kondisi,Keterangan,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date


In [84]:
F01_sent[F01_sent['Tanggal Jatuh Tempo'].isna()]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,Kode Akad Kredit atau Akad Pembiayaan,...,Tanggal Restrukturisasi Akhir,Kode Cara Restrukturisasi,Kode Kondisi,Tanggal Kondisi,Keterangan,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date


In [85]:
DataStorage.download_notebooks_data('./'+prev_pathname+'/f01-'+prev_datem+'.txt', 'kbij/'+prev_pathname, 'f01-'+prev_datem+'.txt')

Blob notebooks/kbij/data202001/f01-202001.txt version LATEST downloaded to ./data202001/f01-202001.txt.


In [86]:
DataStorage.download_notebooks_data('./'+prev_pathname+'/d01-'+prev_datem+'.txt', 'kbij/'+prev_pathname, 'd01-'+prev_datem+'.txt')

Blob notebooks/kbij/data202001/d01-202001.txt version LATEST downloaded to ./data202001/d01-202001.txt.


In [87]:
F01_prev_sent = pd.read_csv('./'+prev_pathname+'/f01-'+prev_datem+'.txt',sep='|',dtype=F01_dtype)
D01_prev_sent = pd.read_csv('./'+prev_pathname+'/d01-'+prev_datem+'.txt',sep='|',dtype=D01_dtype)

In [88]:
print('D01_sent:',D01_sent.shape, 'F01_sent:',F01_sent.shape)
print('D01_prev_sent:',D01_prev_sent.shape, 'F01_prev_sent:',F01_prev_sent.shape)

D01_sent: (119336, 35) F01_sent: (144583, 53)
D01_prev_sent: (112372, 35) F01_prev_sent: (135874, 53)


In [89]:
F01_sent, D01_sent = kbij.checking_data(F01_sent, D01_sent, '2020-05-01', F01_prev=F01_prev_sent, D01_prev=D01_prev_sent)

Check Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data 

[CLEAR]No Previous Not Active Nomor Rekening Fasilitas But Exist in Current Data

[DONE]
Check Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data 

[CLEAR]No Previous Active Nomor Rekening Fasilitas But Not Exist in Current Data

[DONE]
Check Nomor CIF Debitur in D01 and F01 

[CLEAR]All Nomor CIF Debitur in D01 are exist in F01

[DONE]
Check Active Kode Kondisi but Baki Debet = 0 

[CLEAR]No Active Kode Kondisi but Baki Debet = 0

[DONE]
Check Not Active Kode Kondisi but Baki Debet > 0 

[CLEAR]No Not Active Kode Kondisi but Baki Debet > 0

[DONE]
Check Baki Debet > Plafon Awal 

[CLEAR]No Baki Debet greater than Plafon Awal

[DONE]
Check Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Macet 

[CLEAR]No Kode Kualitas Kredit atau Pembiayaan Macet but has no Tanggal Macet

[DONE]
Check Tahun Bulan Data 

F01: ['202002']
D01: ['202002']

[DONE]


In [90]:
D01_sent['Alamat e-mail'][D01_sent['Alamat e-mail'].str.len()>=150]

Series([], Name: Alamat e-mail, dtype: object)

In [91]:
D01_sent['Alamat Tempat Bekerja'][D01_sent['Alamat Tempat Bekerja'].str.len()>=300]

Series([], Name: Alamat Tempat Bekerja, dtype: object)